# Isohrones with GraphHopper to evaluate school accessibility

This notebook is an example how Python code can be used to generate accessibility isochrones to point data with [GrapHopper open source routing engine.](https://github.com/graphhopper/graphhopper)

First commands in the notebook are shell commands which have been tested on Windows. As GrapHopper relies on Java, first start by checking your Java version.

The following code should be run from the Command Prompt, which can be accessed by clicking Windows + R, and then typing "cmd".


``` sh
java -version
pip install maven --user
pip install json
pip install geojason
pip install folium
``` 

This command should return you something like this. Exact output depends on your versions.

```
java version "1.8.0_281"
Java(TM) SE Runtime Environment (build 1.8.0_281-b09)
Java HotSpot(TM) 64-Bit Server VM (build 25.281-b09, mixed mode)
```

Next up if your Java installation is in order, it is time to prepare your GrahHopper instance. Note that you also need git on your machine to run the following commands. You only need to do this the first time.

```sh
git clone git://github.com/graphhopper/graphhopper.git
mkdir graphhopper
cd graphhopper
git checkout 2.0
```

Next download OSM pbf file to your GraphHopper folder. You can find country extracts e.g. from GeoFabrik. Here we download the extract of Jamaica in osm.pbf format which is about 40 mb in size. An alternative, in case the user does not have GUI Wget, is to download this link, and store it in the same folder as graphhopper.

```sh
wget https://download.geofabrik.de/central-america/jamaica-latest.osm.pbf
```

GraphHopper contains config file (config.yml) which defines parameters for the routing engine. Soe for example which vehicle is used for routing and which kind of restrictions should be taken into account. Make a copy of config.yml and modify it accordingly. In our example we are looking at hiking routes as we want to take all paths and routes in to account for school accessibility. Configuration also defines how far from the network the points can be.

Once you have loaded the dataset to your folder, you can start GraphHopper routing engine with the following command:

```sh
graphhopper.sh -a web -i jamaica-latest.osm.pbf -p hike 
```

Go in your browser to the address http://localhost:8989/ and you shoud see a routing demo

Test also this [URL](http://localhost:8989/isochrone?point=18%2C-76.8&time_limit=1800&vehicle=hike&buckets=4)  and it should return you an isochrone in a text format. Now that the API is working, you can import the necessary Python libraries by running the following cell.



In [ ]:
import requests
import json
from geojson import Feature, Polygon, FeatureCollection
import folium # https://python-visualization.github.io/folium/


If some of the imports fail on your machine, you can install the necessary packages e.g. via pip. 

In the API parameters time limit defines the distance traveled in seconds. The number of buckets instead defines how many isochrones are created with the set distance. 

So e.g. with the default settings of the time limit is set to 7200 and the number of buckets is 4, this means that the sample request will generate isochrones for 30, 60, 90 and 120 minutes. 

In [ ]:
startPoint=[18,-76.8]
# Specify the start coordinate. Here we use a random coordinate pair in Jamaica to represent imaginary school location. 

travelTime = 7200
# Specify which time the vehicle should travel. In seconds.

vehicleType = 'hike'
# The vehicle profile for which the route should be calculated.

bucketNo = 4
# Number by which to divide the given time_limit to create buckets nested isochrones of time intervals time_limit-n*time_limit/buckets.

reverseFlow = 'true'
# if true the flow goes from the polygon "inside" to the point.So in our case towards the school


m = folium.Map(location=(startPoint[0],startPoint[1]), zoom_start=12)

api_params = {
    'point': '',
    'time_limit': '',
    'vehicle': '',
    'buckets': '',
    'reverse_flow': ''
}
startCoordinates = ",".join(map(str, startPoint))
api_params.update(point=startCoordinates, time_limit=travelTime, vehicle = vehicleType, buckets = bucketNo, reverse_flow = reverseFlow)

api_url =  'http://localhost:8989/isochrone'

Next we will generate that isochrone with GraphHopper and visualize it with [Folium](https://python-visualization.github.io/folium/). 

In [ ]:
r = requests.get(api_url, params=api_params)
isochrone = r.json()
feature_list = []
i = 0

for i in range(0,bucketNo):
    coordinates = isochrone['polygons'][i]['geometry']['coordinates']

    bucket_feature = Feature(geometry = Polygon(coordinates))   
    feature_list.append(bucket_feature)
    feature_collection = FeatureCollection(feature_list)

folium.GeoJson(feature_collection, name='isochrone').add_to(m)

m

Next up we see an example how a larger CSV file can be converted to isochrones. First import the Pandas library and read in a file with school location coordinates to a variable. 

In [ ]:
import pandas as pd

schools = pd.read_csv('jamaica_schools.csv')

print(schools.head)

Next we will loop through the CSV file and generate an output for each coordinate pair in the file. We wil add id and name fields to each feature.

In [ ]:
api_url =  'http://localhost:8989/isochrone'
api_params = {
    'point': '',
    'time_limit': '',
    'vehicle': '',
    'buckets': '',
    'reverse_flow': ''
}

for index, row in schools.iterrows():

    id = str(row['fid'])
    schoolname = str(row['Name'])
    startY = str(row['Latitude'])
    startX = str(row['Longitude'])
    startPoint = [startY, startX]

    travelTime = 7200
    # Specify which time the vehicle should travel. In seconds.

    vehicleType = 'hike'
    # The vehicle profile for which the route should be calculated.

    bucketNo = 4
    # Number by which to divide the given time_limit to create buckets nested isochrones of time intervals time_limit-n*time_limit/buckets.

    reverseFlow = 'true'
    # If false the flow goes from point to the polygon, if true the flow goes from the polygon "inside" to the point.
    
    startCoordinates = ",".join(map(str, startPoint))
    api_params.update(point=startCoordinates, time_limit=travelTime, vehicle = vehicleType, buckets = bucketNo, reverse_flow = reverseFlow)

    
    r = requests.get(api_url, params=api_params)
    isochrone = r.json()

    try:
        r = requests.get(api_url, params=api_params)
        isochrone = r.json()
        
        feature_list = []
        i = 0
        
        for i in range(0,4):
            coordinates = isochrone['polygons'][i]['geometry']['coordinates']
            
            bucket_feature = Feature(geometry = Polygon(coordinates), properties={'id': id, 'bucket': i, 'name': schoolname})   
            feature_list.append(bucket_feature)
            
            fileName = 'isochrones/' + str(id) + '_iso'
            
            feature_collection = FeatureCollection(feature_list)
            saveFile = open('{0}.geojson'.format(fileName), 'w')
            saveFile.write(str(feature_collection))
            saveFile.close()
            
        
    except:
        pass

This was a very simple notebook example.
